In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import matplotlib.pyplot as plt
import os
from tabulate import tabulate
import sys
import ray

# Initialize Ray
ray.init(ignore_reinit_error=True, local_mode=True)

# Defines the neural network architecture
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        # Input layer (28x28 pixels flattened to 784)
        self.fc1 = nn.Linear(28 * 28, 256)
        # Hidden layers
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        # Output layer (10 classes)
        self.fc4 = nn.Linear(64, 10)
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Flatten the input tensor
        x = x.view(-1, 28 * 28)
        # Pass through layers with ReLU activation and dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        # Output layer (logits)
        x = self.fc4(x)
        return x

# Training function for Ray Tune
def train_model(config):
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torchvision import datasets, transforms
    from torch.utils.data import DataLoader, random_split
    from ray.air import session  # Import session here
    import os

    torch.manual_seed(42)

    # Defined transformations
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    # Loaded the MNIST training dataset
    train_dataset = datasets.MNIST(
        root='./data',
        train=True,
        download=True,  # Ensure dataset is available
        transform=transform
    )

    # Splited into sub-training and validation sets
    sub_train_size = 50000
    valid_size = 10000

    # Ensured total size matches the dataset
    assert sub_train_size + valid_size == len(train_dataset), "Sizes do not match total dataset size."

    # Splited the dataset
    sub_train_dataset, valid_dataset = random_split(
        train_dataset,
        [sub_train_size, valid_size],
        generator=torch.Generator().manual_seed(42)  # For reproducibility
    )

    # Created data loaders
    batch_size = 64
    sub_train_loader = DataLoader(sub_train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialized the model
    model = MNISTNet()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Defined the loss function
    criterion = nn.CrossEntropyLoss()

    # Chose optimizer based on config
    optimizer_name = config["optimizer"]
    if optimizer_name == "SGD":
        optimizer = torch.optim.SGD(
            model.parameters(),
            lr=config["lr"],
            momentum=config["momentum"],
            weight_decay=config["weight_decay"]
        )
    elif optimizer_name == "Adagrad":
        optimizer = torch.optim.Adagrad(
            model.parameters(),
            lr=config["lr"],
            initial_accumulator_value=config["initial_accumulator_value"],
            eps=config["eps"]
        )
    elif optimizer_name == "RMSprop":
        optimizer = torch.optim.RMSprop(
            model.parameters(),
            lr=config["lr"],
            alpha=config["alpha"],
            momentum=config["momentum"]
        )
    elif optimizer_name == "Adam":
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=config["lr"],
            betas=(config["beta1"], config["beta2"])
        )
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    # Lists to store losses
    train_losses = []
    val_losses = []

    # Training loop
    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, labels in sub_train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # Calculate average losses and accuracy
        avg_train_loss = train_loss / len(sub_train_loader.dataset)
        avg_val_loss = val_loss / len(valid_loader.dataset)
        val_accuracy = correct / total

        # Append losses for plotting
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)

        # Save losses after each epoch
        trial_dir = os.getcwd()
        torch.save({
            'train_losses': train_losses,
            'val_losses': val_losses
        }, os.path.join(trial_dir, "losses.pt"))

        # Send metrics to Ray Tune
        session.report(
            {
                "loss": avg_val_loss,
                "accuracy": val_accuracy,
                "training_iteration": epoch + 1  # Report training iteration
            }
        )

# Defined hyperparameter search spaces for each optimizer
# SGD
config_sgd = {
    "optimizer": "SGD",
    "lr": tune.loguniform(1e-4, 1e-1),            # γ (gamma)
    "momentum": tune.uniform(0.0, 0.9),           # μ (mu)
    "weight_decay": tune.loguniform(1e-5, 1e-2),  # τ (tau)
}

# AdaGrad
config_adagrad = {
    "optimizer": "Adagrad",
    "lr": tune.loguniform(1e-4, 1e-1),                    # γ (gamma)
    "initial_accumulator_value": tune.uniform(0.0, 0.1),  # τ (tau)
    "eps": tune.loguniform(1e-10, 1e-8),                  # η (eta)
}

# RMSProp
config_rmsprop = {
    "optimizer": "RMSprop",
    "lr": tune.loguniform(1e-4, 1e-1),   # γ (gamma)
    "alpha": tune.uniform(0.9, 0.99),    # α (alpha)
    "momentum": tune.uniform(0.0, 0.9),  # μ (mu)
}

# Adam
config_adam = {
    "optimizer": "Adam",
    "lr": tune.loguniform(1e-4, 1e-1),   # γ (gamma)
    "beta1": tune.uniform(0.8, 0.99),    # β₁ (beta1)
    "beta2": tune.uniform(0.9, 0.999),   # β₂ (beta2)
}

scheduler = ASHAScheduler(
    metric="accuracy",
    mode="max",
    max_t=5,           # Should match num_epochs
    grace_period=5,    # Set to number of epochs to prevent early stopping
    reduction_factor=2,
    time_attr="training_iteration"
)

# Used CLIReporter and set print_intermediate_tables to False for better notebook compatibility
reporter = CLIReporter(
    parameter_columns=["optimizer", "lr", "momentum", "weight_decay", "alpha", "beta1", "beta2", "initial_accumulator_value", "eps"],
    metric_columns=["loss", "accuracy", "training_iteration"],
    print_intermediate_tables=False
)

# Function to run hyperparameter tuning
def run_tuning(config, num_samples=10, optimizer_name="Optimizer"):
    result = tune.run(
        train_model,
        resources_per_trial={"cpu": 1},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        name=f"{optimizer_name}_tuning"
    )

    # Get the best trial
    best_trial = result.get_best_trial("accuracy", "max", "last")
    print(f"\nBest trial for {optimizer_name}:")
    print("  Hyperparameters: {}".format(best_trial.config))
    print("  Validation Accuracy: {:.4f}".format(best_trial.last_result["accuracy"]))

    # Return best
    return best_trial.config, best_trial.local_path, best_trial.last_result["accuracy"]

# Function to plot training and validation loss curves
def plot_losses(trial_dir, optimizer_name):
    import torch
    import matplotlib.pyplot as plt
    import os

    # Loaded the saved losses
    losses_path = os.path.join(trial_dir, "losses.pt")
    if not os.path.exists(losses_path):
        print(f"No losses.pt found in {trial_dir}")
        return
    losses = torch.load(losses_path)
    train_losses = losses['train_losses']
    val_losses = losses['val_losses']

    # Ploted the losses
    epochs = range(1, len(train_losses) + 1)
    plt.figure()
    plt.plot(epochs, train_losses, 'b-', label='Training Loss')
    plt.plot(epochs, val_losses, 'r-', label='Validation Loss')
    plt.title(f'{optimizer_name} Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# test accuracy function
def evaluate_test_accuracy(config):
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torchvision import datasets, transforms
    from torch.utils.data import DataLoader, random_split

    torch.manual_seed(42)

    # Defined transformations
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    # Loaded the MNIST training dataset
    train_dataset = datasets.MNIST(
        root='./data',
        train=True,
        download=True,  # Ensure dataset is available
        transform=transform
    )

    # Splited into sub-training and validation sets
    sub_train_size = 50000
    valid_size = 10000

    # Ensured total size matches the dataset
    assert sub_train_size + valid_size == len(train_dataset), "Sizes do not match total dataset size."

    sub_train_dataset, _ = random_split(
        train_dataset,
        [sub_train_size, valid_size],
        generator=torch.Generator().manual_seed(42)
    )

    batch_size = 64
    sub_train_loader = DataLoader(sub_train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = datasets.MNIST(
        root='./data',
        train=False,
        download=True,  # Ensure dataset is available
        transform=transform
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = MNISTNet()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer_name = config["optimizer"]
    if optimizer_name == "SGD":
        optimizer = torch.optim.SGD(
            model.parameters(),
            lr=config["lr"],
            momentum=config["momentum"],
            weight_decay=config["weight_decay"]
        )
    elif optimizer_name == "Adagrad":
        optimizer = torch.optim.Adagrad(
            model.parameters(),
            lr=config["lr"],
            initial_accumulator_value=config["initial_accumulator_value"],
            eps=config["eps"]
        )
    elif optimizer_name == "RMSprop":
        optimizer = torch.optim.RMSprop(
            model.parameters(),
            lr=config["lr"],
            alpha=config["alpha"],
            momentum=config["momentum"]
        )
    elif optimizer_name == "Adam":
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=config["lr"],
            betas=(config["beta1"], config["beta2"])
        )
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    num_epochs = 5  # Used the same number of epochs as during tuning
    for epoch in range(num_epochs):
        model.train()
        for images, labels in sub_train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    print(f"Test Accuracy: {test_accuracy:.4f}")
    return test_accuracy  # Return the test accuracy

# Collect best results
results = []

# SGD
best_sgd_config, sgd_trial_dir, val_accuracy_sgd = run_tuning(config_sgd, num_samples=5, optimizer_name="SGD")
plot_losses(sgd_trial_dir, "SGD")
print("SGD Optimizer Test Accuracy:", flush=True)
test_accuracy_sgd = evaluate_test_accuracy(best_sgd_config)
results.append({
    'Optimizer': 'SGD',
    'Validation Accuracy': val_accuracy_sgd,
    'Test Accuracy': test_accuracy_sgd,
    'Best Hyperparameters': best_sgd_config
})

# AdaGrad
best_adagrad_config, adagrad_trial_dir, val_accuracy_adagrad = run_tuning(config_adagrad, num_samples=5, optimizer_name="AdaGrad")
plot_losses(adagrad_trial_dir, "AdaGrad")
print("AdaGrad Optimizer Test Accuracy:", flush=True)
test_accuracy_adagrad = evaluate_test_accuracy(best_adagrad_config)
results.append({
    'Optimizer': 'AdaGrad',
    'Validation Accuracy': val_accuracy_adagrad,
    'Test Accuracy': test_accuracy_adagrad,
    'Best Hyperparameters': best_adagrad_config
})

# RMSProp
best_rmsprop_config, rmsprop_trial_dir, val_accuracy_rmsprop = run_tuning(config_rmsprop, num_samples=5, optimizer_name="RMSProp")
plot_losses(rmsprop_trial_dir, "RMSProp")
print("RMSProp Optimizer Test Accuracy:", flush=True)
test_accuracy_rmsprop = evaluate_test_accuracy(best_rmsprop_config)
results.append({
    'Optimizer': 'RMSProp',
    'Validation Accuracy': val_accuracy_rmsprop,
    'Test Accuracy': test_accuracy_rmsprop,
    'Best Hyperparameters': best_rmsprop_config
})

# Adam
best_adam_config, adam_trial_dir, val_accuracy_adam = run_tuning(config_adam, num_samples=5, optimizer_name="Adam")
plot_losses(adam_trial_dir, "Adam")
print("Adam Optimizer Test Accuracy:", flush=True)
test_accuracy_adam = evaluate_test_accuracy(best_adam_config)
results.append({
    'Optimizer': 'Adam',
    'Validation Accuracy': val_accuracy_adam,
    'Test Accuracy': test_accuracy_adam,
    'Best Hyperparameters': best_adam_config
})


print("\nSummary of Results:", flush=True)
headers = ["Optimizer", "Validation Accuracy", "Test Accuracy", "Best Hyperparameters"]
table = []
for result in results:
    table.append([
        result['Optimizer'],
        "{:.4f}".format(result['Validation Accuracy']),
        "{:.4f}".format(result['Test Accuracy']),
        result['Best Hyperparameters']
    ])
print(tabulate(table, headers=headers, tablefmt="grid"), flush=True)


In [ ]:
Summary of Results:
+-------------+-----------------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------+
| Optimizer   |   Validation Accuracy |   Test Accuracy | Best Hyperparameters                                                                                                                |
+=============+=======================+=================+=====================================================================================================================================+
| SGD         |                0.9699 |          0.9755 | {'optimizer': 'SGD', 'lr': 0.048283097422401555, 'momentum': 0.8824284728982379, 'weight_decay': 0.00018708187816713113}            |
+-------------+-----------------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------+
| AdaGrad     |                0.9657 |          0.9692 | {'optimizer': 'Adagrad', 'lr': 0.02272027337854904, 'initial_accumulator_value': 0.05574957840282573, 'eps': 5.440570610179236e-09} |
+-------------+-----------------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------+
| RMSProp     |                0.9744 |          0.9757 | {'optimizer': 'RMSprop', 'lr': 0.0005303344832494153, 'alpha': 0.9550100132559358, 'momentum': 0.5286150762024874}                  |
+-------------+-----------------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------+
| Adam        |                0.9735 |          0.9758 | {'optimizer': 'Adam', 'lr': 0.0007773168730393929, 'beta1': 0.8167293618180923, 'beta2': 0.9470870377106496}                        |
+-------------+-----------------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------+


# Explaination

## Hyperparameter Tuning
For each optimizer, I searched over a range of hyperparameters to find the best settings that maximize validation accuracy.
This process involved training multiple models with different hyperparameter combinations and selecting the one with the best performance.

# Training and Evaluation
- The models were trained on a large portion of the MNIST dataset (50,000 images).
- Validation accuracy was measured on a separate set (10,000 images) to tune hyperparameters.
- Finally, test accuracy was evaluated on the test dataset (10,000 images) that the model had not seen before.

# Results Interpretation
- **High Accuracies**: All optimizers achieved high accuracies, demonstrating that the neural network architecture is effective for the task.
- **Optimizer Differences**: The slight differences in accuracies can be attributed to how each optimizer adjusts the model's parameters during training.

## Key Takeaways

### Adam Optimizer
- Achieved the highest test accuracy.
- Well-suited for problems with noisy or sparse gradients.
- Balances learning rate adaptation with momentum.

### RMSProp
- Very close performance to Adam.
- Good at handling non-stationary objectives (where the optimal parameters change during training).

### SGD with Momentum
- Performed admirably with a high learning rate and momentum.
- Simple yet effective, especially when tuned properly.

### AdaGrad
- Slightly lower performance.
- May not perform as well on non-convex problems due to aggressive learning rate decay.

### Final Conclusion
- **Model Performance**: The neural network performed exceptionally well in recognizing handwritten digits, achieving over 97% accuracy on the test set.
- **Optimizer Choice**: While all optimizers performed well, Adam provided the best test accuracy in this experiment.
